### * GPU 사용 설정
* by KHW

##### 참고링크 : https://doitgrow.com/28

In [26]:
# import tensorflow as tf

# print()

# print(tf.__version__)  # 설치된 tensorflow 버전 확인

# print(tf.test.is_built_with_cuda()) # 쿠다로 빌드되었는지 확인

# print(tf.test.is_built_with_gpu_support()) # cuda와 같은 gpu로 빌드되었는지 확인

# print(tf.test.gpu_device_name()) # 사용 가능한 gpu 기기들 목록 반환

In [27]:
# # 그래픽카드 인식 확인
# !nvidia-smi

In [28]:
# # 사용 가능한 gpu 확인
# from tensorflow.python.client import device_lib
# device_lib.list_local_devices() 

## Ⅰ. 모델링

In [29]:
# gpu 항상 사용 설정
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

#### 0. 라이브러리 및 데이터 import

In [30]:
import math
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import datetime
import tensorflow as tf
import keras

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import *
from keras.layers import Input
from keras.callbacks import EarlyStopping
# from keras.preprocessing import timeseries_dataset_from_array


from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

from tensorflow.keras.preprocessing import timeseries_dataset_from_array
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Input, LSTM, Dense

In [31]:
rdf=pd.read_csv('output/DL_data.csv')

#### 데이터 전처리
* by KHW

In [32]:
# 결측치 확인
rdf.isnull().sum()

year                     0
month                    0
day                      0
hour                     0
datetime                 0
temp                     0
rainfall                 0
humidity                 0
wind_speed          641698
wind_direction      641698
region                   0
HI                       0
DI                       0
height                   0
가로등                4616952
교육용                9983160
농사용               11189712
산업용                4722144
심야                 4694376
일반용                4616952
임시전력              10754808
주택용                4666800
prev_winter        9095952
dtype: int64

In [33]:
df_all=rdf[:]
print('Number of rows and columns:', df_all.shape)

# 날짜 column 생성
df_all['date'] = pd.to_datetime((df_all['year']*10000 + df_all['month']*100 + df_all['day']).astype(str), format='%Y%m%d')

# region을 넘버링
fact = pd.factorize(df_all['region'])
df_all['region'] = fact[0]

# 여름철 인덱싱
df_all_all=df_all[(df_all['month']==6)|(df_all['month']==7)|(df_all['month']==8)|(df_all['month']==9)]

# 모든 열을 출력한다.
pd.set_option('display.max_columns', None)

df_all

Number of rows and columns: (14831616, 23)


C:\Users\trap1\AppData\Local\Temp\ipykernel_1660\4190721760.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_all['date'] = pd.to_datetime((df_all['year']*10000 + df_all['month']*100 + df_all['day']).astype(str), format='%Y%m%d')
C:\Users\trap1\AppData\Local\Temp\ipykernel_1660\4190721760.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_all['region'] = fact[0]


,year,month,day,hour,datetime,temp,rainfall,humidity,wind_speed,wind_direction,region,HI,DI,height,가로등,교육용,농사용,산업용,심야,일반용,임시전력,주택용,prev_winter,date
0,2010,6,1,0,2010-06-01 00:00:00,16.5,0.0,53.0,NaN,NaN,0,15.589444,60.74355,42.48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2010-06-01
1,2010,6,1,1,2010-06-01 01:00:00,18.1,0.0,49.0,NaN,NaN,0,17.245000,62.73431,42.48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2010-06-01
2,2010,6,1,2,2010-06-01 02:00:00,19.7,0.0,46.0,NaN,NaN,0,18.926667,64.65038,42.48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2010-06-01
3,2010,6,1,3,2010-06-01 03:00:00,20.8,0.0,41.0,NaN,NaN,0,20.006111,65.72772,42.48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2010-06-01
4,2010,6,1,4,2010-06-01 04:00:00,22.2,0.0,36.0,NaN,NaN,0,21.415556,67.04608,42.48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2010-06-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14831611,2022,4,30,19,2022-04-30 19:00:00,10.7,0.0,71.0,2.1,335.0,141,9.679444,52.33503,28.31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-04-30
14831612,2022,4,30,20,2022-04-30 20:00:00,11.0,0.0,65.0,2.2,339.0,141,9.852778,52.99350,28.31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-04-30
14831613,2022,4,30,21,2022-04-30 21:00:00,11.9,0.0,61.0,2.2,338.0,141,10.738333,54.40241,28.31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-04-30
14831614,2022,4,30,22,2022-04-30 22:00:00,13.4,0.0,53.0,2.0,351.0,141,12.179444,56.60598,28.31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-04-30


In [34]:
# prev_winter column이 null인 row 제거 (2010년)
df_all = df_all.dropna(subset= ['prev_winter'])

In [35]:
# column 순서 변경
df_all = df_all[['date', 'datetime', 'year', 'month', 'day', 'temp', 'hour', 'rainfall',
       'humidity', 'wind_speed', 'wind_direction', 'region', 'HI', 'DI',
       'height', '가로등', '교육용', '농사용', '산업용', '심야', '일반용', '임시전력', '주택용',
       'prev_winter']]

In [36]:
# column별 null 값 확인
df_all.isnull().sum()

date                    0
datetime                0
year                    0
month                   0
day                     0
temp                    0
hour                    0
rainfall                0
humidity                0
wind_speed              0
wind_direction          0
region                  0
HI                      0
DI                      0
height                  0
가로등               1439424
교육용               3703560
농사용               4222800
산업용               1484952
심야                1472472
일반용               1439424
임시전력              3990864
주택용               1439424
prev_winter             0
dtype: int64

In [37]:
# 지역 용도 비율의 결측값을 평균으로 처리

for i in range(142):
    df_all.loc[(df_all['region'] == i) & df_all['가로등'].isnull(), '가로등'] = df_all.loc[(df_all['region'] == i), '가로등'].mean()
    df_all.loc[(df_all['region'] == i) & df_all['교육용'].isnull(), '교육용'] = df_all.loc[(df_all['region'] == i), '교육용'].mean()
    df_all.loc[(df_all['region'] == i) & df_all['농사용'].isnull(), '농사용'] = df_all.loc[(df_all['region'] == i), '농사용'].mean()
    df_all.loc[(df_all['region'] == i) & df_all['산업용'].isnull(), '산업용'] = df_all.loc[(df_all['region'] == i), '산업용'].mean()
    df_all.loc[(df_all['region'] == i) & df_all['심야'].isnull(), '심야'] = df_all.loc[(df_all['region'] == i), '심야'].mean()
    df_all.loc[(df_all['region'] == i) & df_all['일반용'].isnull(), '일반용'] = df_all.loc[(df_all['region'] == i), '일반용'].mean()
    df_all.loc[(df_all['region'] == i) & df_all['임시전력'].isnull(), '임시전력'] = df_all.loc[(df_all['region'] == i), '임시전력'].mean()
    df_all.loc[(df_all['region'] == i) & df_all['주택용'].isnull(), '주택용'] = df_all.loc[(df_all['region'] == i), '주택용'].mean()

In [38]:
# 평균 처리 후 NaN값 0으로 처리

df_all.loc[df_all['가로등'].isnull(), '가로등'] = 0
df_all.loc[df_all['교육용'].isnull(), '교육용'] = 0
df_all.loc[df_all['농사용'].isnull(), '농사용'] = 0
df_all.loc[df_all['산업용'].isnull(), '산업용'] = 0
df_all.loc[df_all['심야'].isnull(), '심야'] = 0
df_all.loc[df_all['일반용'].isnull(), '일반용'] = 0
df_all.loc[df_all['임시전력'].isnull(), '임시전력'] = 0
df_all.loc[df_all['주택용'].isnull(), '주택용'] = 0


In [39]:
# index 순서 변경
df_all.reset_index(drop=True, inplace=True)

In [40]:
# 필요없는 column 제거
df_all = df_all.drop(columns=['datetime','year','month','day'])

#### train과 test로 데이터 분리

In [41]:
# Setting
train_split_idx=10000                 # 2020-01-01 23:00:00 행 인덱스의 번호
window_size=1000                      # 과거 100시간 동안 시계열 데이터를 학습 데이터로 사용 / 하이퍼파라미터
future=1                              # 1시간 이후의 타깃 예측

# Features
X_train=df_all.iloc[:train_split_idx-window_size-future,0:]

# Targets
y_train=df_all.iloc[window_size+future:train_split_idx,[1]]  # 'temp' 열

print(X_train.shape,y_train.shape)

(8999, 20) (8999, 1)


In [42]:
df_all.iloc[[train_split_idx-1,train_split_idx,train_split_idx+1]]

,date,temp,hour,rainfall,humidity,wind_speed,wind_direction,region,HI,DI,height,가로등,교육용,농사용,산업용,심야,일반용,임시전력,주택용,prev_winter
9999,2013-08-19,30.5,15,0.0,65.0,1.4,321.0,0,34.990302,81.33675,42.48,3.727941,8.486695,0.0,2.494551,0.614834,64.652375,0.102852,19.920752,4014.0
10000,2013-08-19,30.0,16,0.0,68.0,1.8,311.0,0,34.562978,81.07200,42.48,3.727941,8.486695,0.0,2.494551,0.614834,64.652375,0.102852,19.920752,4014.0
10001,2013-08-19,29.6,17,0.0,66.0,1.8,315.0,0,33.239101,80.17864,42.48,3.727941,8.486695,0.0,2.494551,0.614834,64.652375,0.102852,19.920752,4014.0


In [43]:
# X_test
test_start=train_split_idx-window_size-future   # 테스트 데이터 시작 행
test_end=df_all.shape[0]-window_size-future
X_test=df_all.iloc[test_start:test_end,0:]

# y_test
# label_start= +future        # 테스트 데이터의 첫 번째 타깃 데이터 위치
y_test=df_all.iloc[train_split_idx:,[1]]    # 'temp' 열 선택

print(X_test.shape,y_test.shape)

(5725664, 20) (5725664, 1)


#### 입력 데이터 0~1로 조정

In [44]:
# Feature Scaling
X_train_scaled=X_train.loc[:,'hour':]
X_test_scaled=X_test.loc[:,'hour':]

scaler=MinMaxScaler()
scaler.fit(X_train_scaled.values)
X_train_scaled.loc[:,:]=scaler.transform(X_train_scaled.values)
X_test_scaled.loc[:,:]=scaler.transform(X_test_scaled.values)

## 1차

#### 배치 사이즈 1024
#### epoch 100

In [45]:
# # Mini Batch 크기로 시계열 변환
# train_data=timeseries_dataset_from_array(X_train_scaled,y_train,sequence_length=window_size, batch_size=1024)
# test_data=timeseries_dataset_from_array(X_test_scaled,y_test,sequence_length=window_size, batch_size=1024)

# print(train_data)
# print(test_data)

#### (배치사이즈, 타임스텝, 컬럼수-1)

In [46]:
# for batch in test_data.take(1):
#     inputs,targets=batch

# print("Input:",inputs.numpy().shape)
# print("Target:",targets.numpy().shape)

#### 5층 레이어

In [47]:
# model=Sequential()
# model.add(Input(shape=[100,18]))

# model.add(LSTM(units=64,return_sequences=False))
# model.add(Dense(units=32,activation='linear'))
# model.add(Dense(units=16,activation='linear'))
# model.add(Dense(units=8,activation='linear'))
# model.add(Dense(units=1,activation='linear'))

# model.compile(optimizer='adam',loss='mae',metrics=['mse'])

# model.summary()

In [48]:
# # 모델 훈련
# # with tf.device("/device:GPU:0"):    # GPU를 이용한 학습
# history=model.fit(train_data,epochs=10,validation_data=test_data,verbose=1)

# # 최종 모델 저장
# model.save('./model/origmodel.h5')

# # 손실 함수 그래프
# def plot_loss_curve(history,total_epoch=10,start=1):
#     plt.figure(figsize=(5,5))
#     plt.plot(range(start,total_epoch+1),
#         history.history['loss'][start-1:total_epoch],
#         label='Train')
#     plt.plot(range(start,total_epoch+1),
#         history.history['val_loss'][start-1:total_epoch],
#         label='Validation')
#     plt.xlabel('Epochs')
#     plt.ylabel('mae')
#     plt.legend()
#     plt.show()

# plot_loss_curve(history=history,total_epoch=len(history.history['loss']),start=1)

### 모델 재훈련

In [51]:
# Mini Batch 크기로 시계열 변환
train_data=timeseries_dataset_from_array(X_train_scaled,y_train,sequence_length=window_size, batch_size=1024)
test_data=timeseries_dataset_from_array(X_test_scaled,y_test,sequence_length=window_size, batch_size=1024)

print(train_data)
print(test_data)

<BatchDataset element_spec=(TensorSpec(shape=(None, None, 18), dtype=tf.float64, name=None), TensorSpec(shape=(None, 1), dtype=tf.float64, name=None))>
<BatchDataset element_spec=(TensorSpec(shape=(None, None, 18), dtype=tf.float64, name=None), TensorSpec(shape=(None, 1), dtype=tf.float64, name=None))>


#### (배치사이즈, 타임스텝, 컬럼수-1)

In [52]:
for batch in test_data.take(1):
    inputs,targets=batch

print("Input:",inputs.numpy().shape)
print("Target:",targets.numpy().shape)

Input: (1024, 1000, 18)
Target: (1024, 1)


#### 5층 레이어

In [54]:
# 최종 모델 복원, 요약
loaded_model = tf.keras.models.load_model('./model/model2nd.h5')
loaded_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 64)                21248     
                                                                 
 dense_8 (Dense)             (None, 32)                2080      
                                                                 
 dense_9 (Dense)             (None, 16)                528       
                                                                 
 dense_10 (Dense)            (None, 8)                 136       
                                                                 
 dense_11 (Dense)            (None, 1)                 9         
                                                                 
Total params: 24,001
Trainable params: 24,001
Non-trainable params: 0
_________________________________________________________________


In [56]:
# 모델 재훈련
history=loaded_model.fit(train_data,epochs=20,validation_data=test_data,verbose=1,callbacks=[tf.keras.callbacks.ModelCheckpoint(filepath='./model/model3nd.h5', save_best_only=True, verbose=1)])

# 최종 모델 저장
loaded_model.save('./model/model3nd.h5')

# 손실 함수 그래프
def plot_loss_curve(history,total_epoch=50,start=1):
    plt.figure(figsize=(5,5))
    plt.plot(range(start,total_epoch+1),
        history.history['loss'][start-1:total_epoch],
        label='Train')
    plt.plot(range(start,total_epoch+1),
        history.history['val_loss'][start-1:total_epoch],
        label='Validation')
    plt.xlabel('Epochs')
    plt.ylabel('mae')
    plt.legend()
    plt.show()

plot_loss_curve(history=history,total_epoch=len(history.history['loss']),start=1)

Epoch 1/20
8/8 [==============================] - ETA: 0s - loss: 3.4375 - mse: 18.3181

KeyboardInterrupt: 